# Written Text Recognition, Text Language Translation, and Sentiment Analyzer

## The Question

## The Data

## The Approach

## Our Hypothesis

## Why Bother?

## Basic Setup

In [8]:
!pip install pytesseract
!pip install googletrans
!pip install pytesseract

# adds image processing capabilities
from PIL import Image

# will convert the image to text string
import pytesseract

# translates into preferred language
from googletrans import Translator


## The Data

## Voila!